In [ ]:
pip install PyQt6 pandas pyqtgraph

In [ ]:
import sys
from PyQt6.QtWidgets import (QApplication, QMainWindow, QWidget, QVBoxLayout, 
                            QHBoxLayout, QComboBox, QPushButton, QLabel, 
                            QTableWidget, QTableWidgetItem, QTabWidget, QFileDialog)
from PyQt6.QtCore import Qt
import pandas as pd
import pyqtgraph as pg
import os
from datetime import datetime, timedelta

class MarketDataAnalyzer(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Market Data Analyzer")
        self.setGeometry(100, 100, 1200, 800)
        
        # Store the data
        self.current_df = None
        
        # Available instruments from files
        self.instruments = [
            'AUDUSD', 'BRENT.CMDUSD', 'BTCUSD', 'COCOA.CMDUSD',
            'COFFEE.CMDUSX', 'COPPER.CMDUSD', 'COTTON.CMDUSX',
            'DIESEL.CMDUSD', 'DOLLAR.IDXUSD', 'ETHUSD', 'EURUSD',
            'GAS.CMDUSD', 'GBPUSD', 'LIGHT.CMDUSD', 'NZDUSD',
            'OJUICE.CMDUSX', 'SOYBEAN.CMDUSX', 'SUGAR.CMDUSD',
            'UKGILT.TRGBP', 'USA30.IDXUSD', 'USA500.IDXUSD',
            'USATECH.IDXUSD', 'USDCAD', 'USDCHF', 'USDJPY',
            'USSC2000.IDXUSD', 'USTBOND.TRUSD', 'XAGUSD', 'XAUUSD'
        ]
        
        self.init_ui()
        
    def init_ui(self):
        central_widget = QWidget()
        self.setCentralWidget(central_widget)
        layout = QVBoxLayout(central_widget)
        
        # Create control panel
        control_panel = QWidget()
        control_layout = QHBoxLayout(control_panel)
        
        # Add directory selection button
        self.dir_button = QPushButton("Select Data Directory")
        self.dir_button.clicked.connect(self.select_directory)
        control_layout.addWidget(self.dir_button)
        
        # Directory label
        self.dir_label = QLabel("No directory selected")
        control_layout.addWidget(self.dir_label)
        
        # Instrument selector
        self.instrument_combo = QComboBox()
        self.instrument_combo.addItems(self.instruments)
        control_layout.addWidget(QLabel("Select Instrument:"))
        control_layout.addWidget(self.instrument_combo)
        
        # Time filter selector
        self.timeframe_combo = QComboBox()
        self.timeframe_combo.addItems(['All Data', 'Last 24 Hours', 'Last Week', 'Last Month', 'Last 3 Months', 'Last Year'])
        self.timeframe_combo.currentTextChanged.connect(self.update_chart)
        control_layout.addWidget(QLabel("Time Filter:"))
        control_layout.addWidget(self.timeframe_combo)
        
        # Load data button
        load_button = QPushButton("Load Data")
        load_button.clicked.connect(self.load_data)
        control_layout.addWidget(load_button)
        
        layout.addWidget(control_panel)
        
        # Create tab widget for different views
        self.tab_widget = QTabWidget()
        
        # Chart tab
        self.chart_tab = QWidget()
        chart_layout = QVBoxLayout(self.chart_tab)
        
        # Create plot widget with white background
        self.plot_widget = pg.PlotWidget(background='w')
        self.plot_widget.setLabel('left', 'Price')
        self.plot_widget.setLabel('bottom', 'Time (Hours from Start)')
        
        # Add axis labels
        axis_label_style = {'color': '#000', 'font-size': '14pt'}
        self.plot_widget.setLabel('left', 'Price', units='USD', **axis_label_style)
        self.plot_widget.setLabel('bottom', 'Time', units='Hours', **axis_label_style)
        
        # Enable mouse interaction
        self.plot_widget.setMouseEnabled(x=True, y=True)
        self.plot_widget.showGrid(x=True, y=True)
        
        chart_layout.addWidget(self.plot_widget)
        
        # Add price label widget
        self.price_label = QLabel()
        chart_layout.addWidget(self.price_label)
        
        # Data tab
        self.data_tab = QWidget()
        data_layout = QVBoxLayout(self.data_tab)
        self.table_widget = QTableWidget()
        data_layout.addWidget(self.table_widget)
        
        # Stats tab
        self.stats_tab = QWidget()
        stats_layout = QVBoxLayout(self.stats_tab)
        self.stats_label = QLabel()
        self.stats_label.setAlignment(Qt.AlignmentFlag.AlignTop)
        stats_layout.addWidget(self.stats_label)
        
        # Add tabs to tab widget
        self.tab_widget.addTab(self.chart_tab, "Chart")
        self.tab_widget.addTab(self.data_tab, "Data")
        self.tab_widget.addTab(self.stats_tab, "Statistics")
        
        layout.addWidget(self.tab_widget)
        
        # Store the current directory
        self.current_dir = None

    def filter_data_by_timeframe(self, df, timeframe):
        if timeframe == 'All Data':
            return df
        
        end_date = df['Local time'].max()
        
        if timeframe == 'Last 24 Hours':
            start_date = end_date - timedelta(days=1)
        elif timeframe == 'Last Week':
            start_date = end_date - timedelta(weeks=1)
        elif timeframe == 'Last Month':
            start_date = end_date - timedelta(days=30)
        elif timeframe == 'Last 3 Months':
            start_date = end_date - timedelta(days=90)
        elif timeframe == 'Last Year':
            start_date = end_date - timedelta(days=365)
        
        return df[df['Local time'] >= start_date]

    def update_chart(self):
        if self.current_df is None:
            return
        
        # Filter data based on selected timeframe
        filtered_df = self.filter_data_by_timeframe(self.current_df, self.timeframe_combo.currentText())
        
        # Clear previous plot
        self.plot_widget.clear()
        
        # Create time index (hours from start)
        hours_from_start = [(t - filtered_df['Local time'].iloc[0]).total_seconds() / 3600 
                           for t in filtered_df['Local time']]
        
        # Plot the data
        self.plot_widget.plot(hours_from_start, 
                            filtered_df['Close'].values,
                            pen=pg.mkPen('b', width=2))
        
        # Update price label
        start_price = filtered_df['Close'].iloc[0]
        end_price = filtered_df['Close'].iloc[-1]
        price_change = end_price - start_price
        price_change_pct = (price_change / start_price) * 100
        
        self.price_label.setText(
            f"Start Price: {start_price:.4f} | End Price: {end_price:.4f} | "
            f"Change: {price_change:.4f} ({price_change_pct:.2f}%)"
        )

    def select_directory(self):
        dir_path = QFileDialog.getExistingDirectory(self, "Select Directory with CSV Files")
        if dir_path:
            self.current_dir = dir_path
            self.dir_label.setText(f"Selected: {dir_path}")

    def load_data(self):
        if not self.current_dir:
            self.dir_label.setText("Please select a directory first!")
            return

        try:
            selected_instrument = self.instrument_combo.currentText()
            filename = f"{selected_instrument}_Candlestick_1_Hour_BID_01.01.2020-31.08.2024.csv"
            filepath = os.path.join(self.current_dir, filename)
            
            if not os.path.exists(filepath):
                self.dir_label.setText(f"File not found: {filename}")
                return
            
            # Load data using pandas
            df = pd.read_csv(filepath)
            df['Local time'] = pd.to_datetime(df['Local time'])
            self.current_df = df
            
            # Update chart with time filter
            self.update_chart()
            
            # Update table
            self.update_table(df)
            
            # Update statistics
            self.update_statistics(df)
            
            self.dir_label.setText(f"Loaded: {filename}")
            
        except Exception as e:
            self.dir_label.setText(f"Error loading data: {str(e)}")

    def update_table(self, df):
        # Clear the table first
        self.table_widget.clear()
        self.table_widget.setRowCount(0)
        
        # Set up the table
        self.table_widget.setRowCount(len(df))
        self.table_widget.setColumnCount(len(df.columns))
        self.table_widget.setHorizontalHeaderLabels(df.columns)
        
        # Fill the table
        for i in range(len(df)):
            for j in range(len(df.columns)):
                item = QTableWidgetItem(str(df.iloc[i, j]))
                self.table_widget.setItem(i, j, item)
        
        # Resize columns to content
        self.table_widget.resizeColumnsToContents()
    
    def update_statistics(self, df):
        stats = f"""
        Statistics for {self.instrument_combo.currentText()}:
        
        Price Statistics:
        - Latest Close: {df['Close'].iloc[-1]:.4f}
        - Average Close: {df['Close'].mean():.4f}
        - Maximum Close: {df['Close'].max():.4f}
        - Minimum Close: {df['Close'].min():.4f}
        - Standard Deviation: {df['Close'].std():.4f}
        
        Volume Statistics:
        - Total Volume: {df['Volume'].sum():,.0f}
        - Average Volume: {df['Volume'].mean():.2f}
        - Max Volume: {df['Volume'].max():,.0f}
        
        Date Range:
        - Start Date: {df['Local time'].iloc[0]}
        - End Date: {df['Local time'].iloc[-1]}
        - Total Periods: {len(df)}
        """
        self.stats_label.setText(stats)

if __name__ == '__main__':
    app = QApplication(sys.argv)
    app.setStyle('Fusion')
    window = MarketDataAnalyzer()
    window.show()
    sys.exit(app.exec())